In [0]:
#importing libraries

import tensorflow as tf
import numpy as np
tf.reset_default_graph()

In [0]:
#Bahdanau_attention argumants are . ==> rnn_size (num_units of lstm/gru cell) , 

# encoder_output [batch x max_time x dim] 

# sentence_length ( max _length of sequence in batch)

In [0]:
#let's define encoder first 

batch_size = 4
max_time   = 5
dim        = 12
num_units  = 6
sequence_length = [2,4,5,1]


input_data = np.random.randn(batch_size , max_time , dim).astype(np.float32)

lstm_cell = tf.contrib.rnn.LSTMCell(num_units)

output , last_state = tf.nn.dynamic_rnn(lstm_cell,input_data,sequence_length,dtype=tf.float32)

#input batch x max x dim ==> lstm ==> (batch x max x lstm_dim , batch x lstm_dim )
# return (4, 5, 6) , (4, 6)



# with tf.Session() as sess:
#   sess.run(tf.global_variables_initializer())
#   print(sess.run(last_state[0]).shape)



In [0]:
#now let's Construct the Attention mechanism.


attention_Bahdanau = tf.contrib.seq2seq.BahdanauAttention(
                                                  num_units=num_units,
                                                  memory=output,
                                                  memory_sequence_length=sequence_length,
                                                  dtype=None, 
                                                  name='BahdanauAttention'
                                                 )

#You can test and use it's various methods 

# batch_size = attention_Bahdanau.batch_size
# memory_layer = attention_Bahdanau.memory_layer
# keys=attention_Bahdanau.keys
# values=attention_Bahdanau.values
# alignment_size= attention_Bahdanau.alignments_size
# state_size = attention_Bahdanau.state_size




# will return [batch_size x alignments_size  ] for next time_stamp

# [ batch_size, alignments_size] (alignments_size is memory's max_time).


In [0]:
#let's print the alignment output 

#for that we need two variable , state and query

query_ = tf.get_variable(name='query_dta_1',
                         shape=[batch_size,num_units],
                         dtype=tf.float32,initializer=tf.random_uniform_initializer(-0.01,0.01))

state_ = tf.get_variable(name='state__dta_l',
                         shape=[batch_size,max_time],
                         dtype=tf.float32,initializer=tf.random_uniform_initializer(-0.01,0.01))

In [110]:
#initial_alignments
initial_alignments = attention_Bahdanau.initial_alignments(batch_size,dtype=tf.float32)


#initial_state
initial_state   =    attention_Bahdanau.initial_state(batch_size,dtype=tf.float32)



call = attention_Bahdanau.__call__(query_,state_)

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  alignment , next_state = sess.run(call)
  
  print(alignment.shape,next_state.shape)
  
  #batch_size x max_time  alignment _shape
  #batch_size x max_time  next_state_sgape
  
  
  

(4, 5) (4, 5)


In [0]:
#Now we don't need to do those things , Tensorflow have a wrapper called AttentionWrapper which will do all those things

#attention_wrapper arguments ==> cell , attention_mech , rnn_size 

attention_wrapper = tf.contrib.seq2seq.AttentionWrapper(lstm_cell,
                                                        attention_Bahdanau,
                                                        num_units)

#this will return 

#AttentionWrapperState   batch x num_units
# attention vector       batch x num_units
#current time_stamp      1 , 0 depend of time_staps
#alignments              batch x max_time
# alignment_history
#attention_state         batch x max_time

#     collections.namedtuple("AttentionWrapperState",
#                            ("cell_state", "attention", 
#                                "time", "alignments",
#                             "alignment_history", "attention_state")))

#   """`namedtuple` storing the state of a `AttentionWrapper`.






In [112]:
#let's check and unroll for zero state with timestamp 1

# first we have to define zero_state

zero_s= attention_wrapper.zero_state(batch_size=batch_size,dtype=tf.float32)

wrapper_call = attention_wrapper.__call__(query_,zero_s)
AttentionWrapperState , (cell_state , attention , time , alignments , alignment_history , attention_state) = wrapper_call



with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  AttentionWrapperState , (cell_state , attention , time , alignments , alignment_history , attention_state) =sess.run(wrapper_call)
  

  
  
  print( 'Attention_wrapper_state {} \n '.format(AttentionWrapperState.shape))
  print( 'cell_state {} \n '.format(cell_state))
  print('attention {} \n '.format(attention.shape))
  print('time {} \n ',time)
  print('alignments {} \n '.format(alignments.shape))
  print('alignment_history {} \n ',alignment_history)
  print('attention_state {} \n ',attention_state.shape)
  
  

  
  
  
  
#  Contains:
#     - `cell_state`: The state of the wrapped `RNNCell` at the previous time
#       step.
#     - `attention`: The attention emitted at the previous time step.
#     - `time`: int32 scalar containing the current time step.
#     - `alignments`: A single or tuple of `Tensor`(s) containing the alignments
#        emitted at the previous time step for each attention mechanism.
#     - `alignment_history`: (if enabled) a single or tuple of `TensorArray`(s)
#        containing alignment matrices from all time steps for each attention
#        mechanism. Call `stack()` on each to convert to a `Tensor`.
#     - `attention_state`: A single or tuple of nested objects
#        containing attention mechanism state for each attention mechanism.
#        The objects may contain Tensors or TensorArrays.
#   """



Attention_wrapper_state (4, 6) 
 
cell_state LSTMStateTuple(c=array([[ 1.5111677e-03,  4.0882453e-04,  1.3467169e-03,  1.7782598e-03,
        -1.2312753e-03,  1.5057663e-03],
       [ 2.3543267e-03, -1.9105258e-03,  1.1930702e-03,  7.3114608e-04,
         4.3127441e-04,  1.0493306e-05],
       [-1.5790670e-03, -6.5457175e-04, -3.7221948e-04, -8.0193178e-04,
         5.4796168e-04, -1.5430629e-03],
       [-1.6524252e-03,  3.0396411e-03,  9.2920812e-04,  5.6705176e-04,
        -2.6813590e-03,  3.5907242e-03]], dtype=float32), h=array([[ 7.5694249e-04,  2.0503084e-04,  6.7268399e-04,  8.8943960e-04,
        -6.1760383e-04,  7.5344433e-04],
       [ 1.1791448e-03, -9.5691084e-04,  5.9497467e-04,  3.6539393e-04,
         2.1601986e-04,  5.2492933e-06],
       [-7.8811386e-04, -3.2676206e-04, -1.8669791e-04, -4.0103088e-04,
         2.7398436e-04, -7.7201752e-04],
       [-8.2568289e-04,  1.5217348e-03,  4.6479961e-04,  2.8396057e-04,
        -1.3408703e-03,  1.7910090e-03]], dtype=float32)

In [114]:
#let's feed this time step for next one 

next_unroll = attention_wrapper.__call__(query_,wrapper_call[1])
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  print(sess.run(next_unroll))

  



(array([[ 0.02850083, -0.08838632,  0.03423838, -0.04875825,  0.02433623,
         0.01494354],
       [ 0.0340319 ,  0.14170066, -0.03641218,  0.06206513, -0.02657704,
        -0.15244852],
       [ 0.05387848, -0.05067005, -0.02691523,  0.00859693,  0.07006565,
         0.07642467],
       [ 0.08855353,  0.12874618, -0.12630436,  0.0099225 , -0.07797796,
         0.03092834]], dtype=float32), AttentionWrapperState(cell_state=LSTMStateTuple(c=array([[ 0.00654604, -0.01203434, -0.00615299,  0.0005315 , -0.00387453,
        -0.00444581],
       [ 0.01928671, -0.0066505 ,  0.01987001,  0.01775809,  0.0203748 ,
         0.02813004],
       [-0.00456914,  0.00182934, -0.00227972, -0.02489319,  0.00236064,
        -0.02902747],
       [-0.00715777,  0.00372725,  0.01859315, -0.00267803, -0.01072733,
        -0.00648396]], dtype=float32), h=array([[ 0.00330853, -0.00600193, -0.00309194,  0.00027005, -0.00192373,
        -0.00221843],
       [ 0.00961715, -0.00332273,  0.01001352,  0.00857936